# Testing ways to speed up octree creation

In [ ]:
%load_ext autoreload
%autoreload 2

from octreeStreamOrg import octreeStream

In [ ]:
import datetime

In [ ]:
o = octreeStream('/Users/ageller/VISUALIZATIONS/FIREdata/m12i_res7100/snapdir_600/snapshot_600.0.hdf5', 
                 h5PartKey = 'PartType0', keyList = ['Coordinates', 'Density', 'Velocities'],
                 NNodeMax = 10000, NMemoryMax = 5e4, Nmax=1e5, verbose=1, minWidth=1e-4,
                 path='/Users/ageller/VISUALIZATIONS/octree_threejs_python/WebGL_octreePartition/src/data/junk/octreeNodes/Gas')

In [ ]:
begin_time = datetime.datetime.now()
o.compileOctree()
end_time = datetime.datetime.now()

In [ ]:
print('normal execution time : ',end_time - begin_time)

## Test writing files without pandas

*This is offering a speed up.  I will use this version as the default for now.*

In [ ]:
import sys
sys.path.insert(1, '/Users/ageller/VISUALIZATIONS/octree_threejs_python/python_octreeStream/noPandasTest/')

In [ ]:
%load_ext autoreload
%autoreload 2
from octreeStreamNoPandas import octreeStream as octreeStreamNoPandas

In [ ]:
oP1 = octreeStreamNoPandas('/Users/ageller/VISUALIZATIONS/FIREdata/m12i_res7100/snapdir_600/snapshot_600.0.hdf5', 
                 h5PartKey = 'PartType0', keyList = ['Coordinates', 'Density', 'Velocities'],
                 NNodeMax = 10000, NMemoryMax = 5e4, Nmax=1e5, verbose=1, minWidth=1e-4,
                 cleanDir = True,
                 path='/Users/ageller/VISUALIZATIONS/octree_threejs_python/WebGL_octreePartition/src/data/junk/octreeNodes/Gas')

In [ ]:
begin_timeP1 = datetime.datetime.now()
oP1.compileOctree()
end_timeP1 = datetime.datetime.now()

In [ ]:
print('no pandas execution time : ',end_timeP1 - begin_timeP1)

## Try with multiprocessing

*Starting from the no pandas version.*

*This is somehow WAAAYYY slower!!! (even with multiepl cores in use.) I guess it's because of all the shared memory?*

In [5]:
import sys
import datetime
sys.path.insert(1, '/Users/ageller/VISUALIZATIONS/octree_threejs_python/python_octreeStream/multiprocessingTest/')

In [6]:
%load_ext autoreload
%autoreload 2
from octreeStreamMultiprocessing import octreeStream as octreeStreamMultiprocessing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
oM1 = octreeStreamMultiprocessing('/Users/ageller/VISUALIZATIONS/FIREdata/m12i_res7100/snapdir_600/snapshot_600.0.hdf5', 
                 h5PartKey = 'PartType0', keyList = ['Coordinates', 'Density', 'Velocities'],
                 NNodeMax = 10000, NMemoryMax = 5e4, Nmax=1e5, verbose=1, minWidth=1e-4,
                 cleanDir = True,
                 path='/Users/ageller/VISUALIZATIONS/octree_threejs_python/WebGL_octreePartition/src/data/junk/octreeNodes/Gas')

files will be output to: C:\Users\ageller\VISUALIZATIONS\octree_threejs_python\WebGL_octreePartition\src\data\junk\octreeNodes\Gas


In [ ]:
begin_timeM1 = datetime.datetime.now()
oM1.compileOctree()
end_timeM1 = datetime.datetime.now()

CHECKING NEW NODE {'x': 0, 'y': 0, 'z': 0, 'width': 1000, 'Nparticles': 0, 'id': '0', 'parentNodes': [], 'childNodes': [], 'particles': [], 'needsUpdate': True}
0 50000 0 13712161
starting jobs 1 0 50000 100000.0
joining jobs


In [ ]:
print('multiprocessing execution time : ',end_timeM1 - begin_timeM1)

## Trying cythonize (not working)

https://cython.readthedocs.io/en/latest/src/tutorial/cython_tutorial.html

Had to install Visual Studio's C++ compiler:
https://visualstudio.microsoft.com/visual-cpp-build-tools/


```
$ cp noPandaTest/octreeStreamNoPandas.py cythonTest/octreeStreamCython1.pyx
```

And created a setup.py file.  Then

```
$ python setup.py build_ext --inplace

```

(I will probably want to try to rewrite the code with cython syntax ... but let's see how fast this test is.)


In [ ]:
import sys
sys.path.insert(1, '/Users/ageller/VISUALIZATIONS/octree_threejs_python/python_octreeStream/cythonTest/')

In [ ]:
#%load_ext autoreload
#%autoreload 2
from octreeStreamCython1 import octreeStream as octreeStreamCython1

In [ ]:
oC1 = octreeStreamCython1('/Users/ageller/VISUALIZATIONS/FIREdata/m12i_res7100/snapdir_600/snapshot_600.0.hdf5', 
                 h5PartKey = 'PartType0', keyList = ['Coordinates', 'Density', 'Velocities'],
                 NNodeMax = 10000, NMemoryMax = 5e4, Nmax=1e5, verbose=1, minWidth=1e-4,
                 cleanDir = True,
                 path='/Users/ageller/VISUALIZATIONS/octree_threejs_python/WebGL_octreePartition/src/data/junk/octreeNodes/Gas')

In [ ]:
begin_timeC1 = datetime.datetime.now()
oC1.compileOctree()
end_timeC1 = datetime.datetime.now()

In [ ]:
print('cythonize execution time : ',end_timeC1 - begin_timeC1)

## Trying with numba (not working)

https://numba.pydata.org/numba-doc/dev/user/5minguide.html


```
$ cp octreeStream.py numbaTest/octreeStreamNumba1.py
```

added 
```
@jit(nopython=True)
```
throughout the code


In [ ]:
import datetime
import sys
sys.path.insert(1, '/Users/ageller/VISUALIZATIONS/octree_threejs_python/python_octreeStream/numbaTest/')

In [ ]:
%load_ext autoreload
%autoreload 2
from octreeStreamNumba1 import octreeStream as octreeStreamNumba1

In [ ]:
oN1 = octreeStreamNumba1('/Users/ageller/VISUALIZATIONS/FIREdata/m12i_res7100/snapdir_600/snapshot_600.0.hdf5', 
                 h5PartKey = 'PartType0', keyList = ['Coordinates', 'Density', 'Velocities'],
                 NNodeMax = 10000, NMemoryMax = 5e6, Nmax=1e5, verbose=1, minWidth=1e-4,
                 path='/Users/ageller/VISUALIZATIONS/octree_threejs_python/WebGL_octreePartition/src/data/junk/octreeNodes/Gas')

In [ ]:
begin_timeN1 = datetime.datetime.now()
oN1.compileOctree()
end_timeN1 = datetime.datetime.now()

In [ ]:
print('numba execution time : ',end_timeN1 - begin_timeN1)